In [1]:
import numpy as np
import anndata
import pandas as pd
import scanpy as sc
import scipy
import numpy_indexed as npi
from scipy.sparse import lil_matrix

In [2]:
Path="/home/mgander/moscot-framework_reproducibility/data/mouse_temporal"
ts=['E3.5', 'E4.5', 'E5.25', 'E5.5', 'E6.25', 'E6.5', 'E6.75', 'E7.0', 'E7.25', 'E7.5', 'E7.75', 'E8.0', 'E8.25', 'E8.5a', 'E8.5b', 'E9.5', 'E10.5', 'E11.5', 'E12.5', 'E13.5']

# Load the neirest neighbors and restructure them into a sparse matrix

By running TOME we get csv-files with the neirest neighbors. This accumulates all these nn and shapes it into sparse matrices, that are then considered the analogous to moscots transport matrices.

In [4]:
def cellID_Dict(adata):
    D={}
    
    for i in range(len(adata)):
        D[adata.obs['cellID'][i]]=i
    return(D)

def construct_nn_matrix(ts0, D0, D1, j):
    
    n=pd.read_csv(f'{Path}/TOME_maps_growth_rate_and_driver_genes_analysis/{ts0}/neigbors_{j}.csv')
    c0=list(pd.read_csv(f'{Path}/TOME_maps_growth_rate_and_driver_genes_analysis/{ts0}/early_cells_{j}.csv')['x'])
    c1=list(pd.read_csv(f'{Path}/TOME_maps_growth_rate_and_driver_genes_analysis/{ts0}/late_cells_{j}.csv')['x'])

    D0s={}
    for i in range(len(c0)):
        D0s[i+1]=D0[c0[i]]
    D1s={}
    for i in range(len(c1)):
        D1s[i]=D1[c1[i]]

    N=n.values.copy()
    ar=N.flatten().copy()
    Nn = npi.remap(ar, list(D0s.keys()), list(D0s.values()))
    Nn=Nn.reshape(N.shape)
    return(Nn, D1s)

In [5]:
def TOME_output_to_transport_matrix(A0, A1, ts0):
    D0=cellID_Dict(A0)
    D1=cellID_Dict(A1)
    
    M = lil_matrix((len(A0), len(A1)), dtype=int)

    for j in range(1,501):
        if j%100==0:
            print(j)
        Nn, D1s=construct_nn_matrix(ts0, D0, D1, j)

        for k in range(len(Nn)):
            for ks in range(Nn.shape[1]):
                M[Nn[k,ks], D1s[k]]+=1
    return(M)

In [ ]:
for i in range(19):
    ts0=ts[i]
    ts1=ts[i+1]
    print(ts0)
    
    A0=sc.read(f"{Path}/anndatas/Single_anndatas/adata_{ts0}.h5ad")
    A1=sc.read(f"{Path}/anndatas/Single_anndatas/adata_{ts1}.h5ad")
    
    M=TOME_output_to_transport_matrix(A0, A1, ts0)
    scipy.sparse.save_npz(f'{Path}/TOME_maps_growth_rate_and_driver_genes_analysis/{ts0}.npz', M.tocsr())

E3.5
100
200
300
400
500
E4.5
100
200
300
400
500
E5.25
100
200
300
400
500
E5.5
100
200
300
400
500
E6.25
100
200
300
400
500
E6.5
100
200
300
400
500
E6.75
100
200
300
400
500
E7.0
100
200
300
400
500
E7.25
100
200
300
400
500
E7.5
100
200
300
400
500
E7.75
100
200
300
400
500
E8.0
100
200
300
400
500
E8.25
100
200
300
400
500
E8.5a
100
200
300
400
500
E8.5b
100
200
300
400
500
E9.5
100
